In [27]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [16]:
df = pd.read_excel('Agri-Price.xlsm')

In [17]:
df

,Year,Wheat Price,Corn Price,Soy Price
0,2005,7.51,6.46,16.67
1,2006,9.14,7.47,15.92
2,2007,11.06,12.24,23.20
3,2008,15.22,9.84,24.47
4,2009,9.80,8.92,25.59
5,2010,12.34,13.49,28.43
6,2011,18.05,17.05,33.17
7,2012,21.59,20.82,59.62
8,2013,17.55,16.03,43.65
9,2014,17.86,14.10,37.91


In [18]:
# Feature Engineering: Add previous year's prices as features
df["Wheat_Lag1"] = df["Wheat Price"].shift(1)
df["Corn_Lag1"] = df["Corn Price"].shift(1)
df["Soy_Lag1"] = df["Soy Price"].shift(1)

In [20]:
# Drop rows with NaN (first row due to shifting)
df = df.dropna()

# Define Features (X) and Target (y)
X = df[["Wheat_Lag1", "Corn_Lag1", "Soy_Lag1"]]
y = df[["Wheat Price", "Corn Price", "Soy Price"]]


In [21]:
# Train-test split (2005–2021 for training, 2022 for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1, shuffle=False)

# Scale Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [28]:
# Train XGBoost Models for Each Commodity
models = {}
predictions = {}

for crop in ["Wheat Price", "Corn Price", "Soy Price"]:
    model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1, max_depth=5)
    model.fit(X_train_scaled, y_train[crop])
    
    y_pred = model.predict(X_test_scaled)
    
    models[crop] = model
    predictions[crop] = y_pred
    # Print Model Performance
    print(f"Model Performance for {crop}:")
    print(f"MAE: {mean_absolute_error(y_test[crop], y_pred):.2f}")
    print(f"MSE: {mean_squared_error(y_test[crop], y_pred):.2f}")
    print(f"R² Score: {r2_score(y_test[crop], y_pred):.2f}")


Model Performance for Wheat Price:
MAE: 15.64
MSE: 244.65
R² Score: nan
Model Performance for Corn Price:
MAE: 15.03
MSE: 225.99
R² Score: nan
Model Performance for Soy Price:
MAE: 31.27
MSE: 977.80
R² Score: nan


/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [36]:
# Predict Prices for 2024–2026
future_years = [2024, 2025, 2026]
future_predictions = []

# Start with last known prices
last_known_prices = df.iloc[-1][["Wheat Price", "Corn Price", "Soy Price"]].values

for year in future_years:
    # Prepare input for model
    future_input = np.array(last_known_prices).reshape(1, -1)
    future_input_scaled = scaler.transform(future_input)

    # Predict for each crop
    wheat_pred = models["Wheat Price"].predict(future_input_scaled)[0]
    corn_pred = models["Corn Price"].predict(future_input_scaled)[0]
    soy_pred = models["Soy Price"].predict(future_input_scaled)[0]

    # Append results
    future_predictions.append([year, wheat_pred, corn_pred, soy_pred])

    # Update last known prices for the next iteration
    last_known_prices = [wheat_pred, corn_pred, soy_pred]

# Convert predictions to DataFrame
future_df = pd.DataFrame(future_predictions, columns=["Year", "Wheat_Price_Pred", "Corn_Price_Pred", "Soy_Price_Pred"])

print("\nPredicted Prices for 2024–2026:")
print(future_df)



Predicted Prices for 2024–2026:
   Year  Wheat_Price_Pred  Corn_Price_Pred  Soy_Price_Pred
0  2024         17.548676        16.026945       43.640213
1  2025         16.551397        14.507733       35.447929
2  2026         17.829699        14.390941       34.703922


/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
